In [1]:
from pywgraph import WeightedDirectedGraph, WeightedDirectedEdge, Group
import numpy as np 

In [2]:
r3_group = Group(
    "R^3", np.ones(3), lambda x, y: x * y, lambda x, y: x / y, lambda x: hash(tuple(x))
)
graph = WeightedDirectedGraph.from_dict(
    dict={
        "EUR": {"USD": np.array([0.8, 0.9, 1.1]), "GBP": np.array([1.2, 1.3, 1.4])},
        "USD": {"GBP": np.array([0.7, 0.8, 0.9]), "JPY": np.array([110, 120, 130])},
        "GBP": {"JPY": np.array([150, 160, 170])},
        "JPY": {},
    },
    group=r3_group,
)

In [3]:
graph.find_path("GBP", "GBP")
graph.path_weight(["GBP"])

array([1., 1., 1.])

In [4]:
r_22_matrix = Group(
    name="R2x2 matrices with multiplication", 
    neutral_element=np.identity(2),
    operation=lambda x,y : x*y, 
    inverse_operation=lambda x,y : x/y,
    hash_function=lambda x: hash(tuple(x.flatten()))
)

matrix_graph = WeightedDirectedGraph.from_dict(
    dict={
        "A": {"B": np.array([[1, 2], [3, 4]]), "D": np.array([[2,4], [0.5, 1]])},
        "B": {"C": np.array([[5, 6], [7, 8]])},
        "C": {"D": np.array([[9, 10], [-6, -1.3]])},
        "D": {},
    },
    group=r_22_matrix,
)

In [5]:
def _check_nodes_in_edges(
    nodes: set[str],
    edges: set[WeightedDirectedEdge],
) -> bool:
    edge_nodes = set().union(*[{edge.start, edge.end} for edge in edges])
    return edge_nodes <= nodes

In [7]:
from typing import Any
from pywgraph import Group

def vector_group_multiplication() -> Group: 
    group = Group(
        name="Vectors of dimension 2 with multiplication",
        neutral_element=np.ones(2),
        operation=lambda x,y:x*y,
        inverse_operation=lambda x, y: x/y,
        hash_function=lambda x: hash(tuple(x))
    )
    return group

_array_dict_graph: dict[str, dict[str, Any]] = {
    "A":{
        "B":np.array([1, 2]),
        "C":np.array([3, 4]),
    },
    "B":{
        "C":np.array([-5,1.3]),
        "D":np.array([2, 1]),
    },
    "C":{
        "D":np.array([-1, 1]),
    },
    "D":{},
    "Z":{}
}

def multiplication_graph() -> WeightedDirectedGraph:
    graph = WeightedDirectedGraph.from_dict(
        _array_dict_graph,
        vector_group_multiplication()
    )
    return graph

In [12]:
_check_nodes_in_edges(multiplication_graph().nodes, multiplication_graph().edges)

True